In [ ]:
import $ivy.`io.monix::monix:3.2.2`;
import $ivy.`org.tpolecat::doobie-core:0.9.0`
import $ivy.`org.tpolecat::doobie-hikari:0.9.0`
import $ivy.`co.fs2::fs2-reactive-streams:2.4.4`
import $ivy.`ru.yandex.clickhouse:clickhouse-jdbc:0.2.4`

In [4]:
import ru.yandex.clickhouse.ClickHouseDataSource
import ru.yandex.clickhouse.settings.ClickHouseProperties

val clickHouseProperties = new ClickHouseProperties()

clickHouseProperties.setHost("localhost")
clickHouseProperties.setPort(8123)
clickHouseProperties.setDatabase("docker")
clickHouseProperties.setUser("default")
clickHouseProperties.setPassword("")

val chDataSource = new ClickHouseDataSource("jdbc:clickhouse://localhost:8123/docker")


import ru.yandex.clickhouse.ClickHouseDataSource

import ru.yandex.clickhouse.settings.ClickHouseProperties


clickHouseProperties: ClickHouseProperties = ru.yandex.clickhouse.settings.ClickHouseProperties@1051b863
chDataSource: ClickHouseDataSource = ru.yandex.clickhouse.ClickHouseDataSource@291d4a9f

In [5]:
import doobie.util.transactor.Transactor
import doobie.util.transactor.Transactor.Aux
import monix.eval.Task

import java.util.concurrent.Executors
import scala.concurrent.ExecutionContext
import cats.effect.Blocker

val transactor: Aux[Task, ClickHouseDataSource] = Transactor.fromDataSource[Task](
      chDataSource,
      ExecutionContext.fromExecutor(Executors.newFixedThreadPool(10)),
      Blocker.liftExecutorService(Executors.newCachedThreadPool())
)

import doobie.util.transactor.Transactor

import doobie.util.transactor.Transactor.Aux

import monix.eval.Task


import java.util.concurrent.Executors

import scala.concurrent.ExecutionContext

import cats.effect.Blocker


transactor: Aux[Task, ClickHouseDataSource] = doobie.util.transactor$Transactor$$anon$13@7edf46f6

In [12]:
case class GitHubRecordBaseInfo(
   ghRequestUser: String,
   name: String,
   ownerType: String 
)

defined class GitHubRecordBaseInfo

In [13]:
import doobie.implicits._

object SimpleQueries{
    
    def getCount: doobie.Query0[Int] = {
        
        sql"SELECT COUNT(*) FROM docker.parquet_files".stripMargin.query[Int]
        
    }
    
    def getBaseInfo(limitValue: Int): doobie.Query0[GitHubRecordBaseInfo] = {
        
        sql"""|SELECT ghRequestUser, name, ownerType
              |FROM docker.parquet_files LIMIT $limitValue""".stripMargin.query[GitHubRecordBaseInfo]
        
    }
    
}


import doobie.implicits._


defined object SimpleQueries

In [14]:
import doobie.implicits._
import monix.eval.Task
import fs2.interop.reactivestreams._
import monix.execution.Scheduler.Implicits.global
import monix.reactive.Observable

object SimpleRepository{
    
    def getCount: Task[List[Int]] = {
        
        val publisher = SimpleQueries.getCount
                                     .stream
                                     .transact(transactor)
                                     .toUnicastPublisher
        
        Observable.fromReactivePublisher(publisher).toListL
        
    }
    
    def getBaseInfo(limitValue: Int): Task[List[GitHubRecordBaseInfo]] = {
        
        val publisher = SimpleQueries.getBaseInfo(limitValue)
                                     .stream
                                     .transact(transactor)
                                     .toUnicastPublisher
        
        Observable.fromReactivePublisher(publisher).toListL
        
    }
    
}


import doobie.implicits._

import monix.eval.Task

import fs2.interop.reactivestreams._

import monix.execution.Scheduler.Implicits.global

import monix.reactive.Observable


defined object SimpleRepository

In [15]:
import monix.execution.Scheduler.Implicits.global

import monix.execution.Scheduler.Implicits.global

In [16]:
SimpleRepository.getCount.runSyncUnsafe().head

res15: Int = 6497

In [17]:

val headers: Seq[String] = Seq("ghRequestUser", "name", "ownerType")

val selectResult: List[GitHubRecordBaseInfo] = SimpleRepository.getBaseInfo(5).runSyncUnsafe()

val htmlString: String = 
   s"""
     <table>
       <tr>${headers.map(elem => s"<th>${elem}</th>").mkString}</tr>
       ${selectResult.map(row => s"<tr><td>${row.ghRequestUser}</td><td>${row.name}</td><td>${row.ownerType}</td></tr>").mkString}
     </table>  
    """

kernel.publish.html(htmlString)


ghRequestUser,name,ownerType
neolite,bootstrap,User
neolite,bootstrap-vue,User
neolite,bootstrapper,User
neolite,callback,User
neolite,devart-template,User


headers: Seq[String] = List("ghRequestUser", "name", "ownerType")
selectResult: List[GitHubRecordBaseInfo] = List(
  GitHubRecordBaseInfo("neolite", "bootstrap", "User"),
  GitHubRecordBaseInfo("neolite", "bootstrap-vue", "User"),
  GitHubRecordBaseInfo("neolite", "bootstrapper", "User"),
  GitHubRecordBaseInfo("neolite", "callback", "User"),
  GitHubRecordBaseInfo("neolite", "devart-template", "User")
)
htmlString: String = """
     <table>
       <tr><th>ghRequestUser</th><th>name</th><th>ownerType</th></tr>
       <tr><td>neolite</td><td>bootstrap</td><td>User</td></tr><tr><td>neolite</td><td>bootstrap-vue</td><td>User</td></tr><tr><td>neolite</td><td>bootstrapper</td><td>User</td></tr><tr><td>neolite</td><td>callback</td><td>User</td></tr><tr><td>neolite</td><td>devart-template</td><td>User</td></tr>
     </table>  
    """